In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import keras
import matplotlib.pyplot as plt

%matplotlib inline
sns.set_style('whitegrid')

In [2]:
df_test_features = pd.read_csv('../test_features.csv')

In [3]:
num_test_data = df_test_features.iloc[:,9:]
num_test_data['loudness'] = df_test_features['loudness']
num_test_data['tempo'] = df_test_features['tempo']
num_test_data['duration'] = df_test_features['duration']
cat_test_data = df_test_features[['time_signature', 'key', 'mode']]

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [5]:
cat_test_data['time_signature_cat'] = le.fit_transform(cat_test_data['time_signature'])
cat_test_data['key_cat'] = le.fit_transform(cat_test_data['key'])
cat_test_data['mode_cat'] = le.fit_transform(cat_test_data['mode'])

C:\Users\alex0\AppData\Local\Temp/ipykernel_21252/719706249.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_test_data['time_signature_cat'] = le.fit_transform(cat_test_data['time_signature'])
C:\Users\alex0\AppData\Local\Temp/ipykernel_21252/719706249.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_test_data['key_cat'] = le.fit_transform(cat_test_data['key'])
C:\Users\alex0\AppData\Local\Temp/ipykernel_21252/719706249.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [7]:
num_test = scaler.fit_transform(num_test_data)

In [8]:
predictor = keras.models.load_model('../audio_meta_model')

In [9]:
new_cat_test_data = [cat_test_data.time_signature_cat,cat_test_data.key_cat,cat_test_data.mode_cat]

# Make a prediction

In [12]:
test_dataset =  [num_test] + [new_cat_test_data]

In [13]:
prediction = predictor.predict(test_dataset)

In [14]:
predict_labels = [ label.argmax() for label in prediction]

In [15]:
classes_mapping = {
    0: 'classic pop and rock',
    1: 'dance and electronica',
    2: 'folk',
    3: 'jazz and blues',
    4: 'metal',
    5: 'pop',
    6: 'punk',
    7: 'soul and reggae',
}

In [16]:
predict_label_array = np.vectorize(classes_mapping.get)(predict_labels)

In [17]:
correct_label = pd.read_csv('../test_labels.csv')
correct_label_array = correct_label['genre'].values

In [18]:
from sklearn.metrics import accuracy_score

In [19]:
accuracy = accuracy_score(correct_label_array, predict_label_array)

In [20]:
accuracy

0.4439252336448598

In [21]:
from sklearn.metrics import classification_report
print(classification_report(correct_label_array, predict_label_array))

                       precision    recall  f1-score   support

 classic pop and rock       0.29      0.38      0.32        64
dance and electronica       0.60      0.48      0.53        64
                 folk       0.53      0.63      0.58        73
       jazz and blues       0.12      0.11      0.12        36
                metal       0.81      0.50      0.62        42
                  pop       0.36      0.52      0.43        44
                 punk       0.49      0.50      0.49        40
      soul and reggae       0.51      0.32      0.40        65

             accuracy                           0.44       428
            macro avg       0.46      0.43      0.44       428
         weighted avg       0.47      0.44      0.45       428

